In [3]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=3,
    initial_delay = 1,
    http_status_codes=[429, 500, 503, 504]
)

In [5]:
def get_fee_for_payment_method(payment_method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}"""

    fee_rate = {
        "platinum credit card": 0.3,
        "gold credit card": 0.5,
        "bank transfer": 0.1
    }    

    method = payment_method.lower()
    if method in fee_rate:
        return {"status": "success", "fee_percentage": fee_rate.get(method)}
    else:
        return {" status": "error", "error_message": f"This is not {payment_method} valid in our system"}  

print ("Payment Method Tool Created Successfully")
print ( get_fee_for_payment_method("platinum credit card") )

Payment Method Tool Created Successfully
{'status': 'success', 'fee_percentage': 0.3}


In [6]:
def get_exchange_rate(basecurrency: str, convertcurrency: str)-> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

    exchange_rate = {
        "usd": {
            "eur": 0.93,
            "gbp": 0.81,
            "pkr": 284.5
        }
    }

    base = basecurrency.lower()
    target = convertcurrency.lower()

    if base in exchange_rate and target in exchange_rate.get(base, {}):
        return {"status": "success", "rate": exchange_rate[base][target]}
    else: 
        return { "status": "error", "error_message": f"we doon't support this currency pair {basecurrency} to {convertcurrency}" }


print(" Exchange Rate Tool Created Successfully")
print(get_exchange_rate("usd","pkr"))            

 Exchange Rate Tool Created Successfully
{'status': 'success', 'rate': 284.5}


In [7]:
currency_agent = LlmAgent(
    model = Gemini(
        model_name="gemini-1.5-flash",
        retry_options=retry_config
    ),
    name="CurrencyExchangeAgent",
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.""",
    tools=[
        get_exchange_rate,get_fee_for_payment_method
    ]
)

print("currency_agent created successfully.")

currency_agent created successfully.


In [8]:
runner = InMemoryRunner(agent=currency_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "D:\Google Course\myenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [9]:
result = await runner.run_debug("I want to convert 50 usd in pkr via bank transfer.")


 ### Created new session: debug_session_id

User > I want to convert 50 usd in pkr via bank transfer.


CurrencyExchangeAgent > You will receive 12802.5 PKR.

Here's the breakdown:
A fee of 10.0% (5.0 USD) will be applied to your transaction for using bank transfer. After the fee, the amount remaining for conversion is 45.0 USD.
The exchange rate used is 1 USD = 284.5 PKR.
Therefore, 45.0 USD converts to 12802.5 PKR.


In [12]:
calculator_agent = LlmAgent(
    name="CalculatorAgent",
    model=Gemini(
        model_name="gemini-1.5-flash",
        retry_options=retry_config
    ),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.""",


    code_executor = BuiltInCodeExecutor()
)

In [15]:
professional_currency_agent = LlmAgent(
    name= "ProfessionalCurrencyAgent",
    model=Gemini(
        model_name="gemini-1.5-flash",
        retry_options=retry_config
    ),
    instruction = """You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.""",

    tools=[get_exchange_rate,get_fee_for_payment_method,AgentTool(agent=calculator_agent)]
)


In [16]:
agent_runner = InMemoryRunner(agent= professional_currency_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "D:\Google Course\myenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [17]:
result = await agent_runner.run_debug("I want to convert 50 usd in pkr via gold credit card.")


 ### Created new session: debug_session_id

User > I want to convert 50 usd in pkr via gold credit card.


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "D:\Google Course\myenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


ProfessionalCurrencyAgent > Here's the breakdown of your currency conversion:

*   **Final Converted Amount:** 14153.875 PKR
*   **Detailed Calculation:**
    *   A fee of 0.5% is applied for using a gold credit card.
    *   The fee amount is 0.25 USD (0.5% of 50 USD).
    *   After deducting the fee, the remaining amount is 49.75 USD.
    *   This amount is then converted at an exchange rate of 1 USD = 284.5 PKR.
